In [ ]:
%load_ext autoreload
%autoreload 2
import tmp.set_env


In [ ]:
import numpy as np

from load_dataset import EXTRACT_DIR, get_dataframe, CLIP_VISUAL_EMBS_DIR, COL_NAME_ITEM_ID, COL_NAME_IMAGE_FILE, \
  DATASET_DIR
from scann import scann_ops_pybind
from scann.scann_ops.py.scann_ops import ScannSearcher

from utils import chunker, load_embeddings


In [ ]:
df = get_dataframe("train")


In [ ]:
embeddings_array = load_embeddings(df, CLIP_VISUAL_EMBS_DIR)


In [ ]:
np.save(EXTRACT_DIR + 'train_embeddings_array.npz', embeddings_array)


In [ ]:

"""
https://github.com/google-research/google-research/blob/master/scann/docs/example.ipynb
https://github.com/google-research/google-research/blob/master/scann/docs/algorithms.md
"""

embeddings_array = np.load(EXTRACT_DIR + 'train_embeddings_array.npz')

num_leaves = int(np.sqrt(len(embeddings_array)))
n_neigh = 10


index_builder = scann_ops_pybind.builder(
    embeddings_array,
    num_neighbors=n_neigh,
    distance_measure="dot_product",
)

index_builder = index_builder.tree(num_leaves=num_leaves, num_leaves_to_search=2 * n_neigh)
index_builder = index_builder.score_ah(
            dimensions_per_block=2,
            anisotropic_quantization_threshold=0.2,
        ).reorder(2 * n_neigh)
index: ScannSearcher = index_builder.build()
index.serialize(EXTRACT_DIR + 'index')


In [ ]:
index: ScannSearcher = scann_ops_pybind.load_searcher(EXTRACT_DIR)

In [ ]:


idxs, sims = index.search(embeddings_array[0], final_num_neighbors=3)
